# Exploring the regional response to a warmer world

This notebook allows you to explore the regional response to a warmer world at a series of global **warming levels** (changes in global mean temperature relative to the historical baseline): 1.5˚, 2˚, 3˚, and 4˚C. Warming levels can be used to explore future possible trajectories of any climate variable of interest for a given increase in global mean temperature.

Because warming levels are defined based on amount of global mean temperature change, they can be used to compare possible outcomes across multiple scenarios or model simulations. 

Warming levels are often used in international policy discussions, such as the [Paris Agreement](https://unfccc.int/process-and-meetings/the-paris-agreement/the-paris-agreement) to limit warming to 2˚C.

For a given warming level, we will then use this tool to:

- Examine the range of possibility in maps of regional changes across climate models

- Map day-of-year/time-of-day combinations with especially large increases in temperature, and overlay the locations of power plants to help identify vulnerable assets

## Step 0: Setup

Import libraries needed.

In [ ]:
import climakitae as ck
from climakitae.explore import warming_levels

## Step 1: Explore data

Launch a toolkit view localized projections under varying levels of warming. First, choose a variable, and spatial area of interest. 

You are best off choosing the coarsest time-scale available (monthly) for the visualization, unless you plan to export data at higher time-resolution.

In [ ]:
wl = warming_levels()
wl.choose_data()

### Now retrieve and process the data
The calculate step may take a while to complete. Selecting statistical downscaling will take longer because there are more simulations to work with, and they are at the highest spatial resolution (~10-15min).

In [ ]:
%%time
wl.calculate()

### Next visualize the regional response at a series of global warming levels.
Use the drop down menu to visualize when a specified global warming level in reached for a scenario of interest. Scenarios shown are Shared Socioeconomic Pathways ([SSPs](https://www.sciencedirect.com/science/article/pii/S0959378016300681)): ranging from low (SSP 1-1.9) to high (SSP 5-8.5) emissions trajectories. 

To learn more about the data available on the Analytics Engine, [see our data catalog](https://analytics.cal-adapt.org/data/). 

In [ ]:
wl.visualize()

wl.visualize() step, above, is optional if you want to go directly to extracting the data.

### Extract the data in the snapshots in the panel
Then examine it, plot it yourself some other way, or save it out.

In [ ]:
regional_response = wl.gwl_snapshots

### Extract slices of data centered at warming levels:

In [ ]:
test = wl.sliced_data

**Optional:** Get a feel for how the data is stored, by looking at timeseries for a single grid cell. The 30-year slice is different for each simulation, with some that reach the warming level sooner or later that others.

In [ ]:
if wl.wl_params.downscaling_method == ['Dynamical']:
    out = test.sel(warming_level="2.0").mean(['x','y']).squeeze().to_pandas()
else:
    out = test.sel(warming_level="2.0").mean(['lat','lon']).squeeze().to_pandas()

In [ ]:
out.plot.line(legend=None,figsize=[13,2])

## Step 2: Application

Prepare a map of the changes for a day and time when the largest temperature increases are projected, and overlay the locations of California [power plants](https://cecgis-caenergy.opendata.arcgis.com/datasets/CAEnergy::california-power-plants/explore?location=37.174065,-118.394256,6.93) and [substations](https://cecgis-caenergy.opendata.arcgis.com/datasets/CAEnergy::california-electric-substations/explore?location=37.225884,-119.175779,6.93) on the map to identify vulnerable assets.

We'll start by retrieving some data, making sure that it is hourly time-resolution.

In [ ]:
wl2 = warming_levels()
wl2.choose_data()

Here are some good options for this. You'll want to look over the whole state of California.

In [ ]:
wl2.wl_params.downscaling_method=['Dynamical']
wl2.wl_params.timescale = 'hourly'
wl2.wl_params.resolution = '45 km'
wl2.wl_params.variable='Air Temperature at 2m'
wl2.wl_params.units='degF'

In [ ]:
wl2.calculate()

In [ ]:
my_gwl = 3.0  # 3˚ global warming

In [ ]:
my_data = wl2.sliced_data.sel(warming_level=str(float(my_gwl)))
my_data

Here we select a particular day of the year and time of day to investigate:

In [ ]:
one_day = my_data.isel(time=(my_data.time.dt.dayofyear == 216))
one_hour = one_day.isel(time=(one_day.time.dt.hour == 23))
one_hour

And format the information to be plotted as a difference from historical for a given warming level:

In [ ]:
to_plot = one_hour.mean('time')
to_plot.name = 'degrees warmer'
to_plot

Force the data to be computed and ready to plot. This may take a minute.

In [ ]:
to_plot = ck.load(to_plot)

Load some data that contains locations of power plants:

In [ ]:
import geopandas as gpd

# URLs to shapefiles for example point data from CEC (power plants and substations available)
URLs = {
    'power_plants' : "https://opendata.arcgis.com/api/v3/datasets/4a702cd67be24ae7ab8173423a768e1b_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1",
    'substations' : "https://cecgis-caenergy.opendata.arcgis.com/datasets/CAEnergy::california-electric-substations.geojson?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
}

# Download the desired data
power_plants = gpd.read_file(URLs['power_plants']).rename(columns = {'Lon_WGS84':'lon', 'Lat_WGS84':'lat'})

Finally, prepare the map, and display it with power plant locations overlaid. 

In [ ]:
import cartopy.crs as ccrs
from climakitae.util.utils import read_ae_colormap
cmap = read_ae_colormap(cmap='ae_orange', cmap_hex=False)

gridded = to_plot.plot.pcolormesh(x='lon',y='lat', col='all_sims',
                     transform=ccrs.PlateCarree(),subplot_kws={'projection': ccrs.Orthographic(-100,40)},
                         figsize=[15,5],cmap=cmap)
for ax in gridded.axes.flat:
    ax.coastlines()
    ax.scatter(power_plants.geometry.x, power_plants.geometry.y, transform=ccrs.PlateCarree(),s=0.2,c='k')

## Step 3: Export

To save data as a file, call `export` and input your desired
1) data to export – an [xarray DataArray or Dataset](https://docs.xarray.dev/en/stable/user-guide/data-structures.html), as output by e.g. wl.retrieve()
2) output file name (without file extension)
3) file format ("NetCDF" or "CSV")

To learn more about the file format options, see [getting_started.ipynb](getting_started.ipynb).

In [ ]:
ck.export(to_plot, "my_filename", "NetCDF")